In [1]:
import requests
from pydantic_settings import BaseSettings
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from langgraph.constants import START, END
from langgraph.graph import MessagesState, StateGraph
from datetime import date


class AgentState(MessagesState):
    question: str
    answer: str | None = None
    context: str | None = None
    identify_economic_data: dict | None = None

class Setting(BaseSettings):
    FRED_API_KEY: str

    class Config:
        env_file = "../../.env"
        env_file_encoding = "utf-8"

setting = Setting()

graph_builder = StateGraph(AgentState)

llm = ChatOllama(model="PetrosStav/gemma3-tools:12b", temperature=0)

In [2]:
from langgraph.types import Command
from langchain_core.prompts import ChatPromptTemplate

def supervisor_node(state: MessagesState) -> Command[Literal[*members, "analyst"]]:

def identify_economic(state: AgentState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        당신은 경제 전문가 입니다.
        사용자의 질문에 답을 더욱 잘 하기 위해 필요한 자료를 요청해야 합니다.
        사용자가 제공할 수 있는 정보는 FRED에서 가져온 UNRATE, DTWEXM 정보 입니다.
        어느 정보가 필요한가요?
        다른 답변을 추가하지 않고, 아래와 같은 JSON 형식으로 답변을 제공해주세요.
        {{\"series\": \"UNRATE or DTWEXM\", \"start_date\": \"yyyy-mm-dd\", \"end_date\": \"yyyy-mm-dd\"}}
        """),
        ("human", "질문을 한 시간은 {today} 입니다. {question}\n")
    ])

    chain = prompt | llm
    response: AIMessage = chain.invoke({
        "question": state["question"],
        "today": date.today(),
    })
    return {"identify_economic_data": response.content}

def fred_data(state: AgentState):
    url = f'https://api.stlouisfed.org/fred/series/observations'
    state["identify_economic_data"]
    params = {
        'series_id': date_type,
        'api_key': setting.FRED_API_KEY,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
        'frequency': frequency
    }


In [3]:
graph_builder.add_node('identify_economic', identify_economic)

In [4]:
graph_builder.add_edge(START, "identify_economic")
graph_builder.add_edge("identify_economic", END)

In [5]:
graph = graph_builder.compile()

In [6]:
state = AgentState(
    question="미국 실업률의 추이를 알려주세요."
)
graph.invoke(state)

{'messages': [],
 'question': '미국 실업률의 추이를 알려주세요.',
 'identify_economic_data': '{"data_type": "UNRATE", "start_date": "2020-01-01", "end_date": "2025-03-17"}\n'}